In [1]:
# -*- coding: utf-8 -*-
from miasm.analysis.machine import Machine
from miasm.arch.x86.arch import mn_x86
from miasm.core import parse_asm, asmblock
from miasm.core.locationdb import LocationDB
from miasm.analysis.binary import Container
from future.utils import viewitems
from miasm.loader.strpatchwork import *
from miasm.analysis.data_flow import *
from miasm.jitter.csts import PAGE_READ, PAGE_WRITE

In [2]:
# TODO
# Please add some assemble code to protect 'MOV ECX, 0x23' from dead code elimination

# From Here--------------------------------------------

loc_db = LocationDB()
asmcfg = parse_asm.parse_txt(mn_x86, 32, ''' 
main:
    PUSH EBP
    MOV EBP, ESP
    MOV ECX, 0x23
    MOV ECX, 0x4
    MOV EAX, ECX
    POP EBP
    RET
''', loc_db)

# To HERE --------------------------------------------

In [3]:
loc_db.set_location_offset(loc_db.get_name_location('main'), 0x0)

patches = asmblock.asm_resolve_final(mn_x86, asmcfg)
patch_worker = StrPatchwork()
for offset, raw in patches.items():
    patch_worker[offset] = raw
    
cont = Container.from_string(array_tobytes(patch_worker.s), loc_db=loc_db)
machine = Machine('x86_32')
mdis = machine.dis_engine(cont.bin_stream, loc_db=loc_db)
asmcfg2 = mdis.dis_multiblock(0)
ir_arch = machine.ira(loc_db)
ircfg = ir_arch.new_ircfg_from_asmcfg(asmcfg2)

In [4]:
print('Before Simplification:')
for lbl, irb in viewitems(ircfg.blocks):
    print(irb)

Before Simplification:
b'main':

ESP = ESP + -0x4

@32[ESP + -0x4] = EBP

EBP = ESP

ECX = 0x23

ECX = 0x4

EAX = ECX

ESP = ESP + 0x4

EBP = @32[ESP]

ESP = ESP[0:32] + 0x4

EIP = @32[ESP[0:32]]

IRDst = @32[ESP[0:32]]



In [5]:
deadrm = DeadRemoval(ir_arch)
deadrm(ircfg)

True

In [6]:
print('After Simplification:')

for lbl, irb in viewitems(ircfg.blocks):
    print(irb)

After Simplification:
b'main':

ESP = ESP + -0x4

@32[ESP + -0x4] = EBP



ECX = 0x4

EAX = ECX

ESP = ESP + 0x4

ESP = ESP[0:32] + 0x4

IRDst = @32[ESP[0:32]]



In [12]:
# jitter to confirm that the branch is not actually taken
def code_sentinelle(jitter):
    jitter.run = False
    jitter.pc = 0
    return True

myjit = Machine('x86_32').jitter(loc_db, 'python')
myjit.init_stack()
run_addr = 0x00000000
myjit.vm.add_memory_page(run_addr, PAGE_READ | PAGE_WRITE, array_tobytes(patch_worker.s))
myjit.set_trace_log()
myjit.push_uint32_t(0x1337beef)
myjit.add_breakpoint(0x1337beef, code_sentinelle)
myjit.init_run(run_addr)
myjit.continue_run()

00000000 PUSH       EBP
00000001 MOV        EBP, ESP
00000003 MOV        ECX, 0x23
00000008 MOV        ECX, 0x4
0000000D MOV        EAX, ECX
0000000F POP        EBP
00000010 RET        
